<a href="https://colab.research.google.com/github/jay24rajput/Movie-Reviews/blob/master/Movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRGIMUC/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRGIMUC/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteRGIMUC/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [ ]:
for sentence, label in train_data:
  training_sentences.append(str(sentence.numpy()))
  training_labels.append(label.numpy())

In [ ]:
for sentence, label in test_data:
  testing_sentences.append(str(sentence.numpy()))
  testing_labels.append(label.numpy())

In [ ]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
vocab_size = 10000
max_len = 120
trun_type = 'post'
pad_type = 'post'
oov = '<OOV>'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences=sequences, maxlen=max_len, padding=pad_type, truncating=trun_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(sequences=testing_sequences, maxlen=max_len, padding=pad_type, truncating=trun_type)

In [ ]:
embedding_dim = 16

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_len),
                             tf.keras.layers.GlobalAveragePooling1D(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 102       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    padded,
    training_labels_final,
    epochs = 10,
    validation_data = (test_padded, testing_labels_final)
)

Epoch 1/10
782/782 [==============================] - 4s 6ms/step - loss: 0.5614 - accuracy: 0.7496 - val_loss: 0.4285 - val_accuracy: 0.8110
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.3389 - accuracy: 0.8588 - val_loss: 0.3701 - val_accuracy: 0.8349
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2756 - accuracy: 0.8889 - val_loss: 0.3683 - val_accuracy: 0.8374
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2388 - accuracy: 0.9063 - val_loss: 0.3856 - val_accuracy: 0.8323
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2136 - accuracy: 0.9191 - val_loss: 0.4052 - val_accuracy: 0.8304
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1917 - accuracy: 0.9303 - val_loss: 0.4334 - val_accuracy: 0.8256
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1752 - accuracy: 0.9366 - val_loss: 0.4641 - val_accuracy: 0.8195
Epoch 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received ' ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]
